In [1]:
import pandas as pd
import pandas as pd
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import matplotlib.ticker as ticker
import matplotlib.animation as animation
import pandas as pd
import seaborn as sns
import category_encoders as ce
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.cluster import KMeans


# from keras.utils.np_utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
# from keras.optimizers import RMSprop
# from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# from keras.models import Sequential, load_model, Model
# from keras.applications.vgg16 import VGG16
# from keras.applications.densenet import DenseNet169, DenseNet121
# from keras.applications.resnet import ResNet50

from PIL import Image
import os
from pylab import *
import re
from PIL import Image, ImageChops, ImageEnhance

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
data = pd.read_csv('/kaggle/input/newimagedata/new_data.csv')


In [3]:
data = data.dropna() 


In [4]:
data.head()

,age,gender,isMarried,race,religion,priceRange,district,imagePath
0,20-25,male,False,Sinhala,Buddhism,0-2000,Colombo,1
1,20-25,male,False,Sinhala,Buddhism,0-2000,Kalutara,11
2,20-25,male,False,Tamil,Catholic,0-2000,Gampaha,17
3,20-25,male,False,Tamil,Hindu,0-2000,Kurunegala,7
4,20-25,male,False,Sinhala,Catholic,0-2000,Kurunegala,9


In [5]:
data

,age,gender,isMarried,race,religion,priceRange,district,imagePath
0,20-25,male,False,Sinhala,Buddhism,0-2000,Colombo,1
1,20-25,male,False,Sinhala,Buddhism,0-2000,Kalutara,11
2,20-25,male,False,Tamil,Catholic,0-2000,Gampaha,17
3,20-25,male,False,Tamil,Hindu,0-2000,Kurunegala,7
4,20-25,male,False,Sinhala,Catholic,0-2000,Kurunegala,9
...,...,...,...,...,...,...,...,...
14807,25-30,female,True,Tamil,Hindu,0-2000,Gampaha,23
14808,30-35,male,True,Sinhala,Catholic,2000-4000,Kandy,42
14809,25-30,female,True,Tamil,Catholic,0-2000,Gampaha,25
14810,20-25,male,True,Tamil,Hindu,0-2000,Kandy,2


In [8]:
newdata = pd.get_dummies(data,columns=[ 'age','gender','isMarried','race','religion','priceRange','district'])

In [9]:
newdata

,imagePath,age_20-25,age_25-30,age_30-35,gender_female,gender_male,isMarried_False,isMarried_True,race_Muslim,race_Sinhala,...,religion_Catholic,religion_Hindu,religion_Islam,priceRange_0-2000,priceRange_2000-4000,district_Colombo,district_Gampaha,district_Kalutara,district_Kandy,district_Kurunegala
0,1,1,0,0,0,1,1,0,0,1,...,0,0,0,1,0,1,0,0,0,0
1,11,1,0,0,0,1,1,0,0,1,...,0,0,0,1,0,0,0,1,0,0
2,17,1,0,0,0,1,1,0,0,0,...,1,0,0,1,0,0,1,0,0,0
3,7,1,0,0,0,1,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
4,9,1,0,0,0,1,1,0,0,1,...,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14807,23,0,1,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,0
14808,42,0,0,1,0,1,0,1,0,1,...,1,0,0,0,1,0,0,0,1,0
14809,25,0,1,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,0
14810,2,1,0,0,0,1,0,1,0,0,...,0,1,0,1,0,0,0,0,1,0


In [57]:
# Identify and remove duplicate rows
df_updated = newdata.drop_duplicates()

# Reset the row index
df_updated.reset_index(drop=True, inplace=True)

# Print the DataFrame with reset index
print(df_updated)

      imagePath  age_20-25  age_25-30  age_30-35  gender_female  gender_male  \
0             1          1          0          0              0            1   
1            11          1          0          0              0            1   
2            17          1          0          0              0            1   
3             7          1          0          0              0            1   
4             9          1          0          0              0            1   
...         ...        ...        ...        ...            ...          ...   
2399         48          0          1          0              0            1   
2400         18          1          0          0              0            1   
2401         11          1          0          0              0            1   
2402         38          0          0          1              1            0   
2403          2          1          0          0              0            1   

      isMarried_False  isMarried_True  

In [11]:
df_updated.columns

Index(['imagePath', 'age_20-25', 'age_25-30', 'age_30-35', 'gender_female',
       'gender_male', 'isMarried_False', 'isMarried_True', 'race_Muslim',
       'race_Sinhala', 'race_Tamil', 'religion_Buddhism', 'religion_Catholic',
       'religion_Hindu', 'religion_Islam', 'priceRange_0-2000',
       'priceRange_2000-4000', 'district_Colombo', 'district_Gampaha',
       'district_Kalutara', 'district_Kandy', 'district_Kurunegala'],
      dtype='object')

In [12]:
df_updated.head()

,imagePath,age_20-25,age_25-30,age_30-35,gender_female,gender_male,isMarried_False,isMarried_True,race_Muslim,race_Sinhala,...,religion_Catholic,religion_Hindu,religion_Islam,priceRange_0-2000,priceRange_2000-4000,district_Colombo,district_Gampaha,district_Kalutara,district_Kandy,district_Kurunegala
0,1,1,0,0,0,1,1,0,0,1,...,0,0,0,1,0,1,0,0,0,0
1,11,1,0,0,0,1,1,0,0,1,...,0,0,0,1,0,0,0,1,0,0
2,17,1,0,0,0,1,1,0,0,0,...,1,0,0,1,0,0,1,0,0,0
3,7,1,0,0,0,1,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
4,9,1,0,0,0,1,1,0,0,1,...,1,0,0,1,0,0,0,0,0,1


In [58]:
columns_to_delete = ['priceRange_0-2000', 'priceRange_2000-4000',
       'district_Colombo', 'district_Gampaha', 'district_Kalutara',
       'district_Kandy', 'district_Kurunegala']
df_updated = df_updated.drop(columns_to_delete, axis=1)

In [59]:
df_updated.columns

Index(['imagePath', 'age_20-25', 'age_25-30', 'age_30-35', 'gender_female',
       'gender_male', 'isMarried_False', 'isMarried_True', 'race_Muslim',
       'race_Sinhala', 'race_Tamil', 'religion_Buddhism', 'religion_Catholic',
       'religion_Hindu', 'religion_Islam'],
      dtype='object')

In [61]:
df_updated

,imagePath,age_20-25,age_25-30,age_30-35,gender_female,gender_male,isMarried_False,isMarried_True,race_Muslim,race_Sinhala,race_Tamil,religion_Buddhism,religion_Catholic,religion_Hindu,religion_Islam
0,1,1,0,0,0,1,1,0,0,1,0,1,0,0,0
1,11,1,0,0,0,1,1,0,0,1,0,1,0,0,0
2,17,1,0,0,0,1,1,0,0,0,1,0,1,0,0
3,7,1,0,0,0,1,1,0,0,0,1,0,0,1,0
4,9,1,0,0,0,1,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399,48,0,1,0,0,1,1,0,0,1,0,1,0,0,0
2400,18,1,0,0,0,1,0,1,0,1,0,0,1,0,0
2401,11,1,0,0,0,1,0,1,0,1,0,0,1,0,0
2402,38,0,0,1,1,0,1,0,0,0,1,0,1,0,0


In [62]:
batch_size = 10
total_images = df_updated.shape[0]
num_batches = total_images // batch_size

train_images = []

for batch_idx in tqdm(range(num_batches)):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, total_images)  # Ensure end_idx doesn't go beyond total_images

    batch_images = []
    for i in range(start_idx, end_idx):
        img_path = '/kaggle/input/newimages2/newimages2/' + str(df_updated['imagePath'][i]) + '.jpg'
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224, 3))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img / 255
        batch_images.append(img)

    train_images.extend(batch_images)
    print('Batch', batch_idx, 'processed')

# Process the remaining images (if any) that didn't fit into complete batches
if total_images % batch_size != 0:
    remaining_images = total_images % batch_size
    start_idx = num_batches * batch_size
    end_idx = start_idx + remaining_images

    remaining_batch = []
    for i in range(start_idx, end_idx):
        img_path = '/kaggle/input/newimages2/newimages2/' + str(df_updated['imagePath'][i]) + '.jpg'
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224, 3))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img / 255
        remaining_batch.append(img)

    train_images.extend(remaining_batch)
    print('Remaining images processed')

X = np.array(train_images)

  0%|          | 1/240 [00:00<00:48,  4.96it/s]

Batch 0 processed


  1%|▏         | 3/240 [00:00<00:46,  5.07it/s]

Batch 1 processed
Batch 2 processed


  2%|▏         | 5/240 [00:00<00:34,  6.80it/s]

...

 99%|█████████▉| 237/240 [00:34<00:00,  5.23it/s]

Batch 236 processed


100%|██████████| 240/240 [00:35<00:00,  6.80it/s]

Batch 237 processed
Batch 238 processed
Batch 239 processed


Remaining images processed


In [63]:
X.shape

(2404, 224, 224, 3)

In [64]:
y = np.array(df_updated.drop(['imagePath'],axis=1))
y.shape

(2404, 14)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)


In [48]:
print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (2404, 224, 224, 3)
y shape: (2404, 9)


In [49]:
y_train[1]

array([1, 1, 0, 0, 1, 0, 0, 1, 0], dtype=uint8)

In [50]:
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

In [51]:
y_train[1]

array([1, 1, 0, 0, 1, 0, 0, 1, 0], dtype=int32)

In [52]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='sigmoid'))  # Update to 19 output neurons

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [53]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 220, 220, 16)      1216      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 110, 110, 16)     0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 110, 110, 16)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 106, 106, 32)      12832     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 53, 53, 32)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 53, 53, 32)       

In [54]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [55]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)


Epoch 1/10
34/34 [==============================] - 138s 4s/step - loss: 0.6455 - accuracy: 0.2302 - val_loss: 0.6394 - val_accuracy: 0.3402
Epoch 2/10
34/34 [==============================] - 135s 4s/step - loss: 0.6107 - accuracy: 0.2954 - val_loss: 0.6107 - val_accuracy: 0.5809
Epoch 3/10
34/34 [==============================] - 135s 4s/step - loss: 0.5817 - accuracy: 0.4540 - val_loss: 0.5756 - val_accuracy: 0.4855
Epoch 4/10
34/34 [==============================] - 135s 4s/step - loss: 0.5604 - accuracy: 0.5238 - val_loss: 0.5435 - val_accuracy: 0.5062
Epoch 5/10
34/34 [==============================] - 136s 4s/step - loss: 0.5406 - accuracy: 0.5636 - val_loss: 0.5324 - val_accuracy: 0.5104
Epoch 6/10
34/34 [==============================] - 135s 4s/step - loss: 0.5287 - accuracy: 0.5687 - val_loss: 0.5220 - val_accuracy: 0.5643
Epoch 7/10
34/34 [==============================] - 137s 4s/step - loss: 0.5085 - accuracy: 0.5913 - val_loss: 0.4888 - val_accuracy: 0.5228
Epoch 8/10
34

In [ ]:
columns_to_delete = ['gender_female', 'isMarried_False']
df_updated = df_updated.drop(columns_to_delete, axis=1)